In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
from dqn import DQNAgent

In [2]:
import tensorflow as tf
inp = tf.keras.layers.Input(shape = (8))
x = tf.keras.layers.Dense(512, activation = "relu")(inp)
x = tf.keras.layers.Dense(1024, activation = "relu")(x)
x = tf.keras.layers.Dense(1024, activation = "relu")(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dense(4, activation = "linear")(x)

model = tf.keras.Model(inputs = inp, outputs = x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 512)               4608      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 2,106,372
Trainable params: 2,106,372
Non-trainable params: 0
___________________________________________________

In [3]:
x = DQNAgent(model = model,
                 n_actions = 4,
                 memory_size = 100000, 
                 optimizer = tf.keras.optimizers.Adam(0.001), 
                 gamma = 0.99,
                 batch_size =64,
                 name = "dqn1",
                 target_model_sync = 200,
                 exploration = 0.05)

In [4]:
import gym
envs = [gym.make("LunarLander-v2") for _ in range(8)]

In [5]:
x.train(10000, envs = envs, warmup = 1000, render = False, use_reward_per_episode = False)

1000/1000 [==============================] - 50s 50ms/step - loss: 12.8941 - mean q: 9.0049 - rewards: -0.1605 - t: 54.9352


c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
envs = [gym.make("LunarLander-v2") for _ in range(1)]
x.train(500, envs = envs, warmup = 0, render = True, use_reward_per_episode = False)

 498/1000 [=============>................] - ETA: 12s - loss: 12.7558 - mean q: 12.5886 - rewards: 0.1053 - t: 23.7895

In [9]:
for i in range(len(envs)):
    envs[i].close()